<a href="https://colab.research.google.com/github/padmaakhila/ANN/blob/main/Face_Recognition_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Top-level dataset folder
os.makedirs('dataset/train/person1', exist_ok=True)
os.makedirs('dataset/train/person2', exist_ok=True)
os.makedirs('dataset/val/person1', exist_ok=True)
os.makedirs('dataset/val/person2', exist_ok=True)

print("Folders created successfully!")


Folders created successfully!


In [4]:
import shutil

# Train (7 images each)
p1_train = ['p1_1.jpg','p1_2.jpg','p1_3.jpg','p1_4.jpg','p1_5.jpg','p1_6.jpg','p1_7.jpg']
p2_train = ['p2_1.jpg','p2_2.jpg','p2_3.jpg','p2_4.jpg','p2_5.jpg','p2_6.jpg','p2_7.jpg']

# Val (2 images each)
p1_val = ['p1_8.jpg','p1_9.jpg']
p2_val = ['p2_8.jpg','p2_9.jpg']

for f in p1_train:
    shutil.copy(f, "dataset/train/person1")

for f in p2_train:
    shutil.copy(f, "dataset/train/person2")

for f in p1_val:
    shutil.copy(f, "dataset/val/person1")

for f in p2_val:
    shutil.copy(f, "dataset/val/person2")

print("Files arranged successfully!")


Files arranged successfully!


In [6]:
# Get all files in current directory
files = os.listdir()

# Filter person1 and person2 files
p1_files = sorted([f for f in files if f.startswith("p1_")])
p2_files = sorted([f for f in files if f.startswith("p2_")])

# Train (7) and Val (2)
p1_train = p1_files[:7]    # p1_1 to p1_7
p1_val   = p1_files[7:9]   # p1_8, p1_9

p2_train = p2_files[:7]    # p2_1 to p2_7
p2_val   = p2_files[7:9]   # p2_8, p2_9

In [8]:
import os
import shutil

def safe_move(src, dst_folder):
    dst = os.path.join(dst_folder, src)
    if not os.path.exists(dst):   # move only if file does NOT exist
        shutil.move(src, dst)
        print(f"Moved: {src}")
    else:
        print(f"Skipped (already exists): {src}")

# Move person1 train files
for f in p1_train:
    safe_move(f, "dataset/train/person1")

# Move person2 train files
for f in p2_train:
    safe_move(f, "dataset/train/person2")

# Move person1 val files
for f in p1_val:
    safe_move(f, "dataset/val/person1")

# Move person2 val files
for f in p2_val:
    safe_move(f, "dataset/val/person2")


Skipped (already exists): p1_1.jpg
Skipped (already exists): p1_2.jpg
Skipped (already exists): p1_3.jpg
Skipped (already exists): p1_4.jpg
Skipped (already exists): p1_5.jpg
Skipped (already exists): p1_6.jpg
Skipped (already exists): p1_7.jpg
Skipped (already exists): p2_1.jpg
Skipped (already exists): p2_2.jpg
Skipped (already exists): p2_3.jpg
Skipped (already exists): p2_4.jpg
Skipped (already exists): p2_5.jpg
Skipped (already exists): p2_6.jpg
Skipped (already exists): p2_7.jpg
Skipped (already exists): p1_8.jpg
Skipped (already exists): p1_9.jpg
Skipped (already exists): p2_8.jpg
Skipped (already exists): p2_9.jpg


In [9]:
import os

print("Train Person1:", os.listdir("dataset/train/person1"))
print("Train Person2:", os.listdir("dataset/train/person2"))
print("Val Person1:", os.listdir("dataset/val/person1"))
print("Val Person2:", os.listdir("dataset/val/person2"))


Train Person1: ['p1_7.jpg', 'p1_3.jpg', 'p1_6.jpg', 'p1_1.jpg', 'p1_5.jpg', 'p1_2.jpg', 'p1_4.jpg']
Train Person2: ['p2_1.jpg', 'p2_7.jpg', 'p2_4.jpg', 'p2_2.jpg', 'p2_3.jpg', 'p2_5.jpg', 'p2_6.jpg']
Val Person1: ['p1_8.jpg', 'p1_9.jpg']
Val Person2: ['p2_9.jpg', 'p2_8.jpg']


In [10]:
#CNN Model for Face Recognition (Person1 vs Person2)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (150,150)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,

)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=img_size,
    batch_size=4,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    'dataset/val',
    target_size=img_size,
    batch_size=4,
    class_mode='binary'
)


Found 14 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [11]:
#Build CNN Model
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')   # Binary classification (person1 vs person2)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.4514 - loss: 1.5206 - val_accuracy: 0.5000 - val_loss: 0.7421
Epoch 2/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.3895 - loss: 0.7543 - val_accuracy: 0.5000 - val_loss: 0.7329
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.4500 - loss: 0.7628 - val_accuracy: 0.5000 - val_loss: 0.6953
Epoch 4/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6033 - loss: 0.6745 - val_accuracy: 0.7500 - val_loss: 0.6451
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5986 - loss: 0.6771 - val_accuracy: 0.7500 - val_loss: 0.6128
Epoch 6/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6619 - loss: 0.6202 - val_accuracy: 0.7500 - val_loss: 0.5664
Epoch 7/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.6852 - loss: 0.5606 - val_accuracy: 0.5000 - val_loss: 0.5212
Epoch 8/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.7643 - loss: 0.4783 - val_accuracy: 0.7500 - val_loss: 0.5512
Ep

In [14]:
model.save("face_recognition_cnn_model.keras")
print("Model saved.")


Model saved.
